- <span style="color:#2D3748; background-color:#fff5b1;"> 문제점: 계속 제일 먼저나오는것이 반복적으로 나온다--> 완전한 해결책 X </span>
- 대신 나중에 전처리 과정에서 중복 제거하자..

In [ ]:
# !pip install selenium
# !pip install webdriver-manager

In [ ]:
from selenium import webdriver
import time
import random
import requests
from urllib.request import urlopen
import pandas as pd
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
#desired Capabilities클래스는 테스트를 실행할 수 있는 브라우저/환경의 동작을 정의하는 환경을 설정하는 데 도움이 됩니다.
import urllib.parse
from bs4 import BeautifulSoup as bs
import csv, re, requests


from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# URL

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())
url='https://www.tripadvisor.com/ShowUserReviews-g60763-d107466-r821786251-Frick_Collection-New_York_City_New_York.html?filterLang=ALL'
driver.get(url)

# 쿠키 수락(Optional)

In [ ]:
#쿠키수락 팝업창의 경우는 언제 뜰지모르므로 EC.presence_of_all_elements_located 이용해야한다
# okay_button = WebDriverWait(driver, 30).until(
#         EC.presence_of_all_elements_located([By.CSS_SELECTOR,'#onetrust-accept-btn-handler']))
# okay_button[0].click()
# time.sleep(random.uniform(1,1.7))  

# 크롤링

In [ ]:
review_Title = []
# review_Date= []
review_content=[]
user_country=[]
score=[]
Like=[]

time.sleep(1)
html_doc = driver.page_source
#driver.close()

soup = bs(html_doc,'html.parser')

page =0

while True: #True
    page += 1
    print('page_number: ', page)
    time.sleep(random.uniform(3,4))
    
    whole=driver.find_elements_by_css_selector('div.ui_column.is-9')
    for i in whole:
        
        #리뷰제목
        title = i.find_elements_by_css_selector('span.noQuotes')
        if len(title) > 1:
            review_Title.append(title[0].text.strip())
        else: 
            review_Title.append(title[0].text.strip())
        
        #리뷰내용
        a_1=i.find_elements_by_css_selector('p.partial_entry')
        if len(a_1) > 1:
            review_content.append(a_1[0].text.strip())
        else: 
            review_content.append(a_1[0].text.strip())
        
        #별점
        comment_tags4 = i.find_elements_by_css_selector('span.ui_bubble_rating')
        for heo in comment_tags4:
            Score = heo.get_attribute('class')
            Score = str(Score).split('ui_bubble_rating bubble_')
            Score1 = str(Score[1])
            score.append(Score1)
        
        #좋아요
        comment_tags5 = i.find_elements_by_css_selector('span.numHelp.emphasizeWithColor')
        for l1 in comment_tags5:
            Like.append(l1.text.strip())
        
    
    
    #국가
    comment_tags2 = driver.find_elements_by_css_selector('div.info_text')
    for jee1 in comment_tags2[1::]:
        c=jee1.text
        c_1=c.split('\n')
        if len(c_1) ==1:
            user_country.append('none')
        else:
            user_country.append(c_1[1])
    
    print('review_title 길이: ', f'{len(review_Title)}'+'\n' , 'review_content 길이:' f'{len(review_content)}'+'\n', 'user_country 길이: ', f'{len(user_country)}'+'\n' , 'score 길이: ', f'{len(score)}'+'\n' ,'like 길이: ', f'{len(Like)}' )
    
    #다음 버튼
    next_btn = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located([By.CSS_SELECTOR,'a.nav.next']))

    try:
        next_btn.click()
        time.sleep(random.uniform(2,3))
    
    except:
        print('finish')
        
        break
    time.sleep(1)        

# dataframe으로만들기

In [ ]:
#크롤링 된 데이터 프레임으로 만들기
data1 = {'Title' : review_Title, #title 갯수 안맞음
        'Review' : review_content,
       # 'Date':review_Date,
        'User_country':user_country, #갯수 안맞음
        'Score':score,
        'Like':Like}
df1 = pd.DataFrame(data1)

#장소 추가(바꿔줘야함)
df1['location']='frickcollection'
df1

In [ ]:
driver.quit()

# 후처리

- 중복제거
- User_country열에서 city와 country 분류 (Asian vs. Western 분류 위해)

In [ ]:
df3=df1.drop_duplicates(['Review'])
len(df3)

In [ ]:
#도시와 나라 분리시켜주기
df3['city'] = df3.User_country.str.split(',').str[0]
df3['User_country']=df3.User_country.str.split(',').str[1]
df3.reset_index(drop=False, inplace=True)
df3=df3[['Title','Review','User_country','Score','Like','location','city']]
df3.info()

In [ ]:
#저장
df3.to_csv('review_frickcollection.csv')

# 아시안 나라 직접 분류
- unique 매소드를 활용하여 아시안 나라만 분류함. 직접 체크하고 변수에 리스트로 저장해줌
- 이외의 나라에 대해서는 일일히 직접 리뷰를 보며 인덱스로 추가해주는 작업을 거쳤음 (너무 복잡하여 해당 과정은 생략함)

In [ ]:
asia_country=[' 埼玉県',' 대한민국',' Japan',' Taipei',' Shiga',' Funabashi',' 東京都',' 東',' 福岡',' 日本',' Kanagawa',' fukuoka',' South Korea',' Tokyo',' China', ' Hong Kong',' Shizuoka',' Mongolia', ' Korea', 
              ' Macau',' Taiwan','日本','愛知','神奈川', ' South Korea',' ハワイ州',' 神奈川県','서울','부산','경기도']
asia_city=['Korea','South Korea','korea','桑名市','Tokyo Prefecture','アメリカ','千葉','鹿児島','埼玉県','TOKYO','アメリカ','YOKOHAMA','長崎','熊本市','シンガポール','ソウル','埼玉',
           '北海道','大阪','神奈川','坂井市','富山市','八戸市','seoul','incheon','South Korea','Seoul','Incheon','Japan','okinawa',
           'China','越谷','tokyo','Macau','Shizuoka','神戸','osaka','OSAKA','Osaka','品川区','市川市','Tokyo' ,'fukuoka', 'Osaka', 
           'Hong Kong','新潟','東京','京都','長崎','横浜','さいたま','横浜市','川崎市','熊本','埼玉県','埼玉','福岡','愛知','茨城',
           '大阪','名古屋','名古屋','千葉','埼玉','福岡市','横浜','岐阜','倉敷','横浜','長野','神奈川','Nagoya','千葉','神戸','岐阜県',
           '秋田','tokyo','埼玉','神奈川','대한민국','シンガポール','YOKOHAMA','北海道', '北海道','西宮','ニューヨーク','千葉','Hsinchu','北海道','石川',
           'ソウル','TOKYO','アメリカ','さいたま', '柏', '岐阜','Taipei City', 'Kanagawa','Setagaya', 'Seoul', 'Tokyo', '横浜市', 'Japan', 'Akita', '弘前', 'Tokyo Prefecture', 'Nagoya', 'Noto-cho','Kanagawa Prefecture', '新潟', 'Kagoshima', 'Busan’,’Niigata Prefecture', 'Sasebo', '仙台市', 'Sendai', 'Osaka', 'Fukuoka Prefecture', 'Fukuoka', '日本', 'Hong Kong', 'Chubu', 'Sapporo', 'Okayama Prefecture', 
           'Ota', '横浜', 'Yokohama', 'Taichung', '愛知', '千葉', 'Ueda', 'Kanazawa', 'Incheon’, ’Kawasaki', 'Ｔｏｋｙｏ', 'Meguro', '札幌', 'Kobe', 'Kawaguchi', '神奈川', 'Yokosuka-shi', '北海道', 
           'Toyohashi', 'Shitara-cho', 'Osaka Prefecture', 'Urayasu', '八王子市', 'Matsubushi-machi', 'Kabukicho', 'Hiroshima', '福岡', 'TOKYO', 'Suginami', 'Hirosaki', 'Kyoto', '大阪', 
           'Toyonaka', 'Maibara', 'Tochigi', 'Shibuya’,’Yanai', '仙台', 'Moriguchi', 'Mizuho', '大阪府', 'Yao', '神戸市', 'Shiga Prefecture', 
           'Kuwana', '和歌山市', 'Matsuyama', 'Miyazaki', 'Iwate Prefecture', 'Ninomiya-machi', '岡山市', 'Utsunomiya', 'Toyono-cho', 'Hokkaido', 'Konan', 'Gifu', 'Chiba', 
           'Tsuruoka', 'Chiba Prefecture', '熊本', 'Niigata', 'Fujisawa', '三重県', 'Nara Prefecture', 'San Francisco', 'Itabashi', 'Matsusaka', 
           'Fujimi', 'shiga', 'Matsue', 'Naha', '宮崎', 'nakanoku tokyo', 'Toyota', 'kanagawa', '那覇市', 'Kochi Prefecture', '広島', 
           'Hiroshima Prefecture', 'Shinjuku', '埼玉県東松山', 'Hachinohe', '京都',  'Okinawa City', '埼玉', 'Iki', 'Nabari', '盛岡市', 'Miki', 'tokyo',
           'Kumamoto', 'Otsuki', 'Hamamatsu-shi', 'Hokuto', 'Matsudo', 'Fuchu', 'Mito', 'Taipei', 'Antarctica', 'Saitama', 'Mejiro', '京都市', 'Kochi',
           'Changchun', 'Koto-ku', 'Komae', 'Noda', 'Gimhae', '留萌市', '東京23区', 'Kanawaga', 'Nerima', 'Ichikawa', 'Beppu', '岩手県', 'Ise', 'Seika-cho', '岐阜県',
           'Fuji',  '愛知県岡崎市', '서울', 'Shikokuchuo', 'Oita', '대전', '東京都調布市', 'Okayama’,’ fukuoka.jp', 'Ehime Prefecture', 'Kashiwa', '山梨', 
           '埼玉県入間市’ , ’Izunokuni', 'Koto', 'Yokohama-shi', '明石市', '名古屋市', 'Akashi’,’ Kagawa Prefecture', 'Nagasaki', 'Tokorozawa', '東京都', 'Uji', 'Kamisato-machi', 'Hakataekimae', 
           'Funabashi', 'Shanghai', '川崎','okinawa', '兵庫県', '静岡', 'Sumida-ku', '名古屋', 'Asaka','Saitama Prefecture', 'Nishinomiya', 'japan', 'Kumano', '川崎市','神奈川県',
           'Setagaya-ku', '東京都品川区', '長野', '埼玉県', 'バンコク', 'Mobara','Tokushima', 'Asahikawa', '一宮市', 'hirosimasi', '茨城', 'Toshima','Hiratsuka', 'Iriomote-jima', '石川県',
           'kamaishi', 'shizuoka', '東大阪市', 'Kawagoe', '伊豆地方', '神奈川、日本', '沖縄県', '神奈川県茅ヶ崎市', '徳島','Narashino', '船橋', '和歌山', 'Mino', 'Ushiku', 'osaka', '茨城県', '神戸', '長野市', 'Shizuoka', '三重', '滋賀', '奈良', '兵庫', '横須賀', 'kumakiti', '岡山',  'さいたま', '柏', '岐阜', 'Kanagawa', '宝塚市', '広島市', '千葉市', 'nagoya', '東', '浜松', '愛知県', '京都府', '長崎', '松山市', '金沢', '千葉県茂原市', '千葉県', '桑名', 'South Korea', 'Yokosuka', 'Shenzhen', 'China', 'Beijing', 'Uruma','Fussa', 'Macau', 'Pohang', 'Hefei', 'Ayase', 'Guangzhou','HOKKAIDO', 'Seongnam', 'Kiyokawa-mura', 'Hiki-gun', 'Tottori Prefecture', 'Shizuoka Prefecture', 'Chofu', 'Yamanashi', 'Chuo', 'Kyotanabe', 'Hakodate', 'New Taipei', 'Hadano','Tomigusuku', 'Koganei', 'Tachikawa', 'Kumamoto Prefecture', '高知','Kita', 'Obihiro', 'Ibaraki', 'Matsumoto', 'Odawara','Hyogo Prefecture', 'Kurashiki', 'Kyoto Prefecture', 'Kanuma','Nishio', 'Shinagawa', 'Kusatsu', 'Taipei City', 'Koriyama',
           'Gwangju', 'Takizawa', 'Neyagawa', 'Izumi', '高槻市', 'Okinawa Prefecture', 'Kunitachi', 'Hakone-machi', 'Kashiwara', 'Takarazuka', 'Kawanishi', 'Kaohsiung', 'Wakayama Prefecture', 'Nara', 'Higashihiroshima', 'Soka', '平塚市', 'Ishigaki', '大阪市西成区', 'Otsu-shi', '鳴門市', '島根', 'Yongin', 'Shiroi', 'Tochigi Prefecture', 'Toyama', 'Nishitokyo', 'Edogawa', 'Otsu', 'Kahoku', 'Sakai', 'Sumida', 'Noshiro', 'Ibaraki Prefecture', 'Sagamihara', 'Kanto', 'Yachiyo', 'Nagano Prefecture', 'Sendagaya', 'Asakusa', 'Ibaraki-machi', 'Sanda', 'Toyooka', 'Nasu-gun', 'Kahoku-cho', '四日市市', 'Oita Prefecture', 'Aioi', 'Nonoichi', 'Chigasaki','Azumino', 'Yamato', 'Tsu', '鎌倉', 'Chita', 'Komaki', 'Hirakata','Taito', 'Agui-cho', 'Ginza', 'Sangenjaya', '大和町', 'saitama','Ikoma', 'Nirasaki', '福岡市', 'Kameido', 'Morioka', 'Gimpo','千葉県千葉市', '長野県', 'Higashiibaraki-gun', '中標津', 'Nakano', 'Chino','Isesaki', 'Tsukuba', '逗子市', '奈良県', '世田谷区', 'Ogikubo', 'yokohama','Gimcheon', 
           'Watari-cho', 'おおいた','Dongguan', 'Wuxi', 'Insadong', 'Qingdao', 'Sagamihara', 'Kasama','Shanghai', 'Nagareyama','Changwon', 'Tottori', 'Annaka', 'Kawasaki-shi', 'Kitaazumi-gun', 'Azabujuban', 'Himeji', 
           'shanghai', 'Kamakura', 'Suita', 'Nagano', 'Ebetsu', 'Chiryu', '熊本県', '小山市', 'Shinshiro', '高松市', 'Suwon', 'Shiroishi', 'Takatsuki', 'Cheongju', 'Toyama Prefecture', 
           'Yugawara-machi', 'Zushi', 'Nanao', 'Wenshan', 'Geoje', 'Joetsu', 'Okazaki', 'Tomisato', 'Fukutsu', 'Gyeongsangnam-do', 'Taiwan', 'Koshigaya', 'Wakayama', 'Musashino', 'Komoro', 
           '和歌山県', '愛媛', 'Dazaifu', 'マレーシア', '静岡県', 'Higashiosaka', 'Kofu', '松山', '제천', '登別市', 'Chiyoda-ku', 'Settsu', 'Wonju', '富山市', 'Gunpo','Shimizu-cho', 'さいたま市', '札幌市', 
           'Takamatsu', 'Tama', 'Seosan', 'Gyeonggi-do', 'Asia', 'TYO', 'Daegu', 'Fujiyoshida', 'Miyoshi','Pyeongtaek', '所沢', 'Takanezawa-machi', 'Kure', '宝塚市', '千葉市', 'Kawasaki', '東', '浜松', '愛知県', 
           '京都府', '長崎', '松山市', '金沢','静岡県','愛知','金沢','横浜','茨城','広島','浜松市','埼玉県','千葉県茂原市','兵庫','山口','岩手','東京','アメリカ','千葉','神奈川','福岡','Hokkaido','熊本','那須','青森','suzuka','シンガポール',
           'Funabashi','ソウル','福岡','고양시','栃木','Osaka']

for A_index, A in zip(df3.index, df3['User_country']):
    if A in asia_country:
        asia_indx.append(A_index)
        
for A1_index, A1 in zip(df3.index, df3['city']):
    if A1 in asia_city:
        asia_indx.append(A1_index)

asia_indx= set(asia_indx)
asia_indx=list(asia_indx)
print(len(set(asia_indx)))

A_review=df3.iloc[asia_indx]
print(len(A_review))

# 구글번역 API

In [ ]:
from googletrans import Translator
#from google_trans_new import google_translator  
# Translator 클래스 객체 선언 (translator라는 변수명은 마음대로 정해주면 됨)
def google_translate(word):
    translator = Translator()
    translator.raise_Exception = True
    try:
        translated = translator.translate(word, dest='en').text
    except Exception as err:
        translator = Translator()
        translator.raise_Exception = True
        translated = translator.translate(word, dest='en').text
        
    return translated

In [ ]:
eng_Review=[]
for i in A_review['Review']:
    res=google_translate(i)
    eng_Review.append(res)

eng_Title=[]
for j in A_review['Title']:
    resu=google_translate(j)
    eng_Title.append(resu)

print(len(eng_Review))
print(len(eng_Title))

A_review['Review']=eng_Review
A_review['Title']=eng_Title
A_review

In [ ]:
#번역 후 아시아 저장
A_review.to_csv('./Asian/asian_frick(225개).csv')